In [1]:
from sklearn.grid_search import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import log_loss, mean_squared_error as mse, r2_score 
from sklearn.metrics.scorer import make_scorer

C:\Users\jiguo\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\jiguo\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
import kagglegym
env = kagglegym.make()
o = env.reset()
excl = [env.ID_COL_NAME, env.SAMPLE_COL_NAME, env.TARGET_COL_NAME, env.TIME_COL_NAME]
col = [c for c in o.train.columns if c not in excl]

O = pd.read_hdf('../input/train.h5')
d_mean= O[col].median(axis=0)

ymean_dict = dict(o.train.groupby(["id"])["y"].median())

X_train=(O[col])[O.timestamp <= 905]
y_train=O.y[O.timestamp <= 905]
X_test=(O[col])[O.timestamp > 905]
y_test=O.y[O.timestamp > 905]
X_train=X_train.fillna(d_mean)
X_test=X_test.fillna(d_mean)

In [ ]:
estimator=RandomForestRegressor()
param_grid = {'n_estimators':[100], 'max_features':['auto','square'], 'max_depth':[4]}
my_scorer=make_scorer(mse, greater_is_better=False)
model = GridSearchCV(
    estimator=estimator, param_grid=param_grid, n_jobs=-1, cv=5, verbose=20, scoring=my_scorer)
model.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] max_features=auto, n_estimators=100 .............................
[CV] .. max_features=auto, n_estimators=100, score=-0.000746 -514.4min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 514.4min remaining:    0.0s


[CV] max_features=auto, n_estimators=100 .............................


In [ ]:
estimator=XGBRegressor()
param_grid = {'max_depth':[3], 'learning_rate':[.1], 'n_estimators': [100], 'subsample':[.5,1], 'colsample_bytree':[.5,1],\
'base_score':[.5], 'seed':[0]}
my_scorer=make_scorer(mse, greater_is_better=False)
model = GridSearchCV(
    estimator=estimator, param_grid=param_grid, n_jobs=1, cv=5, verbose=20, scoring=my_scorer)
model.fit(X_train, y_train)

In [ ]:
param_grid = {'n_estimators':[500, 600, 700, 800, 900, 1000], 'max_features':['auto','sqrt'], 'max_depth':[4,5,6,7,8,9]}
for n_est in param_grid['n_estimators']:
    for max_feat in param_grid['max_features']:
        for max_d in param_grid['max_depth']:
            model=RandomForestClassifier(n_estimators=n_est, max_features=max_feat, max_depth=max_d)
            score=CVScore(model=model, my_score=log_loss, X_train=train_X, y_train=train_y)
            print('n_est={}, max_feat={}, max_d={}, score={}'.format(n_est, max_feat, max_d, score))

In [ ]:
param_grid = {'eta':[.05, .1], 'num_round':[300,400,500], 'subsample':[.8], 'colsample_bytree':[.5,.85], \
              'max_depth':[7,8,9], 'seed':[2017]}
for eta in param_grid['eta']:
    for subsample in param_grid['subsample']:
        for colsample_bytree in param_grid['colsample_bytree']:
            for max_depth in param_grid['max_depth']:
                for seed in param_grid['seed']:
                    model=xgbClass(colsample_bytree=colsample_bytree, eta=eta, eva_metric='mlogloss', \
                                       subsample=subsample, max_depth=max_depth, seed=seed,\
                                       objective='multi:softprob', num_class=3)
                    score=CVScore(model=model, my_score=log_loss, X_train=train_X, y_train=train_y)
                    print('eta={}, subsample={}, colsample_bytree={}, max_depth={}, seed={}, score={}'.\
                          format(eta, subsample, colsample_bytree, max_depth, seed, score))